In [ ]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import concurrent.futures

from tqdm import tqdm
from pathlib import Path

In [ ]:
PROJECT_ROOT = Path().cwd().parent.parent
PATH_TO_APK_CATALOG = PROJECT_ROOT / "apks" / "latest.csv"

apk_catalog = pd.read_csv(PATH_TO_APK_CATALOG)

# Format the date columns
apk_catalog["dex_date"] = pd.to_datetime(apk_catalog["dex_date"], format="%Y-%m-%d %H:%M:%S")
apk_catalog["vt_scan_date"] = pd.to_datetime(apk_catalog["vt_scan_date"], format="%Y-%m-%d %H:%M:%S")
display(apk_catalog.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25242698 entries, 0 to 25242697
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   sha256        object        
 1   sha1          object        
 2   md5           object        
 3   dex_date      datetime64[ns]
 4   apk_size      int64         
 5   pkg_name      object        
 6   vercode       float64       
 7   vt_detection  float64       
 8   vt_scan_date  datetime64[ns]
 9   dex_size      int64         
 10  markets       object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(5)
memory usage: 2.1+ GB


None

In [3]:
apk_catalog.head(10)

,sha256,sha1,md5,dex_date,apk_size,pkg_name,vercode,vt_detection,vt_scan_date,dex_size,markets
0,0000003B455A6C7AF837EF90F2EAFFD856E3B5CF49F5E2...,9C14D537A7ADB4CFC43D291352F73E05E0CCDD4A,3EDFC78AB53521942798AD551027D04F,2016-04-05 17:58:46,10386469,com.zte.bamachaye,1.210000e+02,0.0,2016-06-15 15:26:44,4765888,anzhi
1,0000014A634DB98F85038B833A8DFC50D5FB13A464E0B2...,C3EBEC52C9388BF67479FF1385A56C59B3E39E81,0A146750FB447CF3859C9CB659AB04F1,2014-08-20 13:58:10,3537486,com.tanersenel.onlinetvizle,1.600000e+01,0.0,2016-11-20 10:26:31,3170096,PlayDrone
2,000001A94F46A0C3DDA514E1F24E675648835BBA5EF3C3...,C0444D784685EFE5F6D9F28683B24B5873E509CB,EC82771AE018B93AD784A1FD2B625216,1980-01-01 00:00:00,52469861,com.firstchoice.myfirstchoice,1.206145e+06,0.0,2021-03-17 08:02:21,9201656,play.google.com
3,000002B63FAD4B030787F6DE4081DC1E12325026EB7DDA...,DD723B32EDD9F70AADBD66846621967157DF9BD4,985E601C17F0A9346590AE92A5AD664E,1980-01-01 00:00:00,4300370,com.deperu.sitiosarequipa,1.000000e+04,0.0,2017-12-03 06:50:28,4211104,play.google.com
4,000003D3981DC548A772A30D688F424CFB88561A63A2DD...,6AE9F138F7E0C63E5D58CC7E82FB05A50F041637,F4789023733E41EE883208ACBC956020,1980-01-01 00:00:00,12958838,com.safetravels.safetravelsmain,4.001250e+05,0.0,2019-06-25 08:55:42,7813520,play.google.com
5,00000439A3FFA123C3F9BC45E5E821351B1A5C276871B3...,375D6FFD167BB5E7E2935B1B927F87D8E44A9AB4,9283C74DD8356C18BB6D94B88B8FDD9B,2011-10-25 02:30:56,1044597,bmthx.god102409paperi,6.000000e+00,1.0,2014-04-27 06:30:31,105660,appchina
6,0000049D8911607971A3336DE5CF36F4799D679D6BB9EF...,433307FF815B8BA4A88BC3EE3EC2C912FFBA063B,B03BEEAFB1975881F11395C4F5F6E2ED,1980-01-01 00:00:00,3161615,com.bmi.calculatorplus,1.100000e+01,0.0,2020-07-13 07:37:27,4000432,play.google.com
7,0000062E08AE15E2D2D3DB3B56A6DE0BCB4F5DB52931F7...,40B1E2CCAD4AF009F5DC516655953A71BBDE4786,F0FBA597A4225C6DCDA9703C8B22DFCE,1981-01-01 01:01:02,55999935,br.com.winker.wl.villesainteanne,1.721317e+09,0.0,2024-10-08 05:49:17,9782760,play.google.com
8,0000072CEFB104E974B04E7C634C91D0AEBC7BA9826B7D...,35CF646C9B61199B7021D89392F89EA3967F61BE,FDD693C02B13DE4146AF9BD975856329,1981-01-01 01:01:02,12634646,com.kitapevi,6.000000e+00,0.0,2025-02-10 07:11:19,7590552,play.google.com
9,0000090C2169EFF7E9A889CA1953CF39C887E0FD23A9BE...,E86150BBF222D038F607BA91AD5E6E70C4C86C2A,39DC3F83F0F1E15A791A3964738388E1,2014-08-21 23:38:28,2541606,eremeev.dev.montecarloradio,8.000000e+00,0.0,2016-05-19 18:52:41,2302164,PlayDrone


In [4]:
apk_catalog["markets"].value_counts().head(20)

markets
play.google.com              20887736
anzhi                         1027489
appchina                       961962
PlayDrone|play.google.com      622778
play.google.com|PlayDrone      621690
VirusShare                     366632
PlayDrone                      146075
mi.com                          91761
fdroid                          73457
slideme                         45251
angeeks                         42543
play.google.com|appchina        39830
appchina|play.google.com        39666
appchina|anzhi                  30115
anzhi|appchina                  29999
1mobile                         29801
praguard                        10173
unknown                          9900
VirusShare|appchina              8053
appchina|VirusShare              7925
Name: count, dtype: int64

In [5]:
# Filter out APKs from VirusShare that are older than 2022-01-01 and have a size less than 10MB
filtered_malware_apk_catalog = apk_catalog[apk_catalog["markets"] == "VirusShare"]
filtered_malware_apk_catalog = filtered_malware_apk_catalog[filtered_malware_apk_catalog["vt_scan_date"] > "2022-01-01"]
filtered_malware_apk_catalog = filtered_malware_apk_catalog[filtered_malware_apk_catalog["apk_size"] < 10_000_000]

print("Filtered Malware APK Catalog size: ", filtered_malware_apk_catalog.shape[0])
filtered_malware_apk_catalog.head(10)

Filtered Malware APK Catalog size:  31239


,sha256,sha1,md5,dex_date,apk_size,pkg_name,vercode,vt_detection,vt_scan_date,dex_size,markets
406,000109A075DC3AFA88C45523A3EDF2039177386C58A489...,DC1A76AF8F7D8A04975FB77BB0AE90672493EBFE,372B3D882C739CDFD1C878E58D28E56A,2020-01-02 15:39:34,7662392,com.antbyte.facepay_miaoshua,4.0,11.0,2022-10-13 11:53:48,4423460,VirusShare
781,000201C8DCB16350F49B60927F32ED99A97389CA6A2F38...,BA0DEA70515EAB1ECAD3285079CD5638810BB598,5CFE2E7AC33E4E5D900D782A5D79C9E7,2020-04-16 03:39:58,1466679,com.lingfenggu.mud,1.0,1.0,2022-02-27 05:01:28,195572,VirusShare
4114,000AA7AC8DE37606FCCD3EA00E5EF9238D550514D61CAD...,EA7A4BA4969CA793B261F133484D25BA0966E19F,AF624A0DB11DB0FE153DFD43BBF566D6,2017-03-31 13:52:42,9421510,com.idddx.lwp.inkandwashseasons3d,44.0,27.0,2022-10-12 20:47:07,5720444,VirusShare
4345,000B460A9F0180C8B83272F0E8AEBE5CDD936E3CE85A77...,EC2EDE70AE4E9E175BA9059E482FD12C2F243639,E3A22FE076375AD65AE93A2882315B1C,2011-12-02 20:54:46,2633063,xcxin.filexpert,128.0,2.0,2022-03-29 17:23:53,3242028,VirusShare
7438,00131C7DF08A13384C37DC166C38DAF84044F0921D6920...,7F7BF30C9AEB86AC990659CDA7A8AE3440A83FDD,D73C4BE66CBC09C963C1E809D71FA3F0,2011-07-27 14:38:50,672007,com.softmimo.android.mileagetracker,135.0,2.0,2022-09-15 13:57:43,198132,VirusShare
7605,00138BEA574298CAF953F4496299F9FCA11E6F414D0603...,5BDEB5A03E4790F03D85096CFAC1D6A486418589,06F4F2ABE9726499F0DDE03D3B279501,2016-06-27 18:02:48,9114429,com.mrocker.aunt,16.0,13.0,2022-09-15 13:43:53,4449868,VirusShare
7809,00142B3D4E52F1411CFBCE064970A536F8190962081809...,AFEEEBF168DD6BA12F67B8E34057DD326C053941,6B6147EC357C970439A77ADE3663DFE4,2010-09-28 23:40:56,1427537,com.cyxb.fishin2go,37.0,1.0,2022-08-03 10:20:56,253944,VirusShare
7954,001486CB026E8CEA1D4212482D851F6E64EEBE796F0429...,78976237DA37516E042354FEA89EEA8CB621EBA8,928A09862A036654E0D899C049CCA897,1980-01-01 00:00:00,6371814,com.zkkj.haidiaoyouque,7.0,2.0,2022-09-15 13:39:02,5641180,VirusShare
8119,0014F2839924EFBE237158F750F0C462D531AFE8BB988B...,A78EF8FD329E303BBBCB60321C84A9363F9DC1CB,3985909CBEF3C93176B9F28FA8701ECB,2017-08-28 14:20:26,3787385,com.yzwl.ssmd,100.0,9.0,2022-09-15 13:40:02,2487624,VirusShare
8732,00167D61D71A8A0F050B7AADBB2124E6F4048C9C36E2DE...,B1074B48E675298F1066E3F890A781ED5FD2F65C,5864D78A449F72763BFFD31117C48869,2013-06-21 17:02:10,2578671,com.phoneplayer.snipertraining,7.0,23.0,2022-08-03 09:47:11,999236,VirusShare


In [6]:
# Filter out APKs from Google Play Store that have 0 detections in VirusTotal, are older than 2022-01-01 and have a size less than 10MB
filtered_benign_apk_catalog = apk_catalog[apk_catalog["markets"] == "play.google.com"]
filtered_benign_apk_catalog = filtered_benign_apk_catalog[filtered_benign_apk_catalog["vt_scan_date"] > "2022-01-01"]
filtered_benign_apk_catalog = filtered_benign_apk_catalog[filtered_benign_apk_catalog["vt_detection"] == 0]
filtered_benign_apk_catalog = filtered_benign_apk_catalog[filtered_benign_apk_catalog["apk_size"] < 10_000_000]

print("Filtered Malware APK Catalog size: ", filtered_benign_apk_catalog.shape[0])
filtered_benign_apk_catalog.head(10)

Filtered Malware APK Catalog size:  1832502


,sha256,sha1,md5,dex_date,apk_size,pkg_name,vercode,vt_detection,vt_scan_date,dex_size,markets
20,0000103AF1B89F26FF77209541EFF57DA9989137194440...,2DD82AD3DC5CBA1BFF01BD0714C59F75F7F3F65D,08094F033E8F442882592E0F509A024C,1981-01-01 01:01:02,6650179,com.dudesolutions.sdcm01,38.0,0.0,2022-04-30 13:33:14,2754764,play.google.com
35,00001B280911A2CB409F2D9A8799E115D96360E05065E3...,25FB6B4E5E1DB8B7ECC3B7BAA7DA9966F4674FAD,CA8C016D7D2B293F21E93FF3D6EC17FD,1981-01-01 01:01:02,9606534,com.eraradio.radiostanicebih,7.0,0.0,2023-05-13 04:50:34,8959012,play.google.com
42,00001ECA82980CC28717F94671EC8BE049D66EC5BC8F2B...,B0D6324F7CF36F619F2AA90439EC3BC8E85D0036,742A36DA734148549931C1426EF6EDFA,1981-01-01 01:01:02,6421982,co.com.qvo.hera,7.0,0.0,2023-03-25 08:11:58,8990436,play.google.com
56,00002446EE3E5C0596316AE5F0408FE67ECB5AC1E9C6A6...,99C923ACDD3574BA1C5F654BCEA24763A3B64242,3CA63B8F9AD89CCD12A9EF723759BE35,1981-01-01 01:01:02,8287161,com.zenoti.dermaenvy,301.0,0.0,2023-03-02 14:11:38,6321692,play.google.com
65,00002A679AA1BA4CDEF68E31418E6714E99AC4B63AE0F0...,2716D5393718B329C7F8F4BCBECF4B046AD4DFE8,F02869EF2E352F3A4DC3FE6DC14ED959,1981-01-01 01:01:02,9343536,com.drivermonitoring,9.0,0.0,2023-04-25 08:37:06,9518644,play.google.com
76,000032D5B11CB0B00DA0A45F08812A576B7A926209B8ED...,D17F2DF9E78D933CCD84E2388C09EF7B85ADCCC4,B0A9F9A21A3338965FF475957ADAFADF,1981-01-01 01:01:02,3382093,com.kasm.Fill,2.0,0.0,2022-04-05 07:06:50,4480732,play.google.com
81,000036AC32CD1D71E70DA2B0E16EC007050976EDBE1B69...,69B8FC268DC22F798CF3E68C08F24C5A3FF5E3F9,049288B09472FE07DC004E86DA9C1C80,1980-01-01 00:00:00,4207678,alice.spiral.nebula.theme,4.0,0.0,2023-01-26 15:29:36,4562652,play.google.com
98,000040A9B9EBC7FCD250B9A115C015114BCD77E10A94D1...,D8083681C407DE26B4D619848A166E56F203EB10,8936267DED158831B5E9213879CF8AEE,1981-01-01 01:01:02,1234522,com.navayuga.bazaar,1.0,0.0,2022-07-13 08:01:50,975012,play.google.com
105,000045DA15CB7D7F02C21F2A8E1D5478967BA7788B3C0F...,D5218966711DB21DDC2596AC0820C33A64694EE9,2E612DC174EDF451E8C706E0BA10E251,1981-01-01 01:01:02,7389070,com.aerilys.baseball.companion.two,8.0,0.0,2023-10-03 07:29:16,8482908,play.google.com
108,000047EBE8AF204F8E39CC9CFC314EF96EC37EC0A145B3...,041377816BE37F841887DEF9C232BFA22CA83414,956D222BC1665E6176570ED14EA4476D,1981-01-01 01:01:02,2853154,it.briefingenova.big,17.0,0.0,2022-10-28 11:42:30,2111676,play.google.com


In [7]:
apk_catalog.describe()

,dex_date,apk_size,vercode,vt_detection,vt_scan_date,dex_size
count,25242698,2.524270e+07,2.523931e+07,2.519384e+07,25193842,2.524270e+07
mean,1991-10-21 14:49:27.705531776,2.173876e+07,4.271792e+07,9.051790e-01,2020-03-14 13:51:32.793174272,5.976932e+06
min,1980-01-01 00:00:00,3.128000e+03,-2.092867e+09,0.000000e+00,2009-10-20 20:29:46,0.000000e+00
25%,1980-01-01 00:00:00,5.384983e+06,5.000000e+00,0.000000e+00,2018-08-21 08:22:22.249999872,3.449716e+06
50%,1981-01-01 01:01:02,1.206256e+07,2.900000e+01,0.000000e+00,2020-11-23 11:50:21,6.642902e+06
75%,2014-01-10 23:22:29,2.780771e+07,5.050000e+02,0.000000e+00,2022-05-09 09:30:12,8.551972e+06
max,2107-12-31 23:00:00,3.530780e+09,2.147484e+09,5.800000e+01,2025-05-21 05:02:29,5.791811e+08
std,NaN,3.071325e+07,2.616052e+08,3.639709e+00,NaN,3.428553e+06


In [ ]:
def download_apk(apk_sha256, output_dir, api_key):
    """
    Download the APK file from AndroZoo using the provided SHA256 hash.
    """
    
    url = f"https://androzoo.uni.lu/api/download?apikey={api_key}&sha256={apk_sha256}"
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        output_path = os.path.join(output_dir, f"{apk_sha256}")
        with open(output_path, "wb") as apk_file:
            for chunk in response.iter_content(chunk_size=8192):
                apk_file.write(chunk)
        print(f"Downloaded: {apk_sha256}")
    else:
        print(f"Failed to download {apk_sha256}: {response.status_code}")
        
def download_apks_parallel(apk_dataframe, output_dir, api_key, num_samples=50, max_workers=10):
    """
    Download a random selection of APKs in parallel.
    
    Parameters:
    - apk_dataframe: DataFrame containing APK information with sha256 hashes
    - output_dir: Directory to save downloaded APKs
    - num_samples: Number of APKs to download
    - max_workers: Maximum number of parallel downloads
    
    Returns:
    - List of successfully downloaded APKs
    """
    # Create output directory if it doesnt exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Select random samples from the dataframe
    selected_apks = apk_dataframe.sample(n=min(num_samples, len(apk_dataframe)))
    sha256_list = selected_apks['sha256'].tolist()
    
    # Track successful downloads
    successful_downloads = []
    failed_downloads = []
    
    # Define a wrapper function to track progress
    def download_with_tracking(sha256):
        try:
            download_apk(sha256, output_dir, api_key)
            return (sha256, True)
        except Exception as e:
            return (sha256, False, str(e))
    
    # Use ThreadPoolExecutor for parallel downloads
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all download tasks
        future_to_sha256 = {executor.submit(download_with_tracking, sha256): sha256 for sha256 in sha256_list}
        
        # Process results as they complete with a progress bar
        with tqdm(total=len(sha256_list), desc="Downloading APKs") as pbar:
            for future in concurrent.futures.as_completed(future_to_sha256):
                result = future.result()
                if isinstance(result, tuple) and len(result) == 2:
                    sha256, success = result
                    if success:
                        successful_downloads.append(sha256)
                    else:
                        failed_downloads.append(sha256)
                else:
                    sha256, success, error_msg = result
                    failed_downloads.append((sha256, error_msg))
                pbar.update(1)
    
    # Print summary
    print("\nDownload Summary:")
    print(f"  Successful: {len(successful_downloads)}/{len(sha256_list)}")
    print(f"  Failed: {len(failed_downloads)}/{len(sha256_list)}")
    
    return successful_downloads

In [ ]:
DOT_ENV_PATH = PROJECT_ROOT / ".env"

load_dotenv(DOT_ENV_PATH)

ANDROZOO_API_KEY = os.getenv("ANDROZOO_API_KEY")
NUM_SAMPLES = 10_000
MAX_WORKERS = 100
BENIGN_DIR = PROJECT_ROOT / "apks" / "20k" / "benign_apks"
MALWARE_DIR = PROJECT_ROOT / "apks" / "20k" / "malware_apks"

if ANDROZOO_API_KEY is None:
    raise ValueError("ANDROZOO_API_KEY is not set in the environment variables.")

# Doanload the malware APKs
download_apks_parallel(
    filtered_malware_apk_catalog,
    output_dir=MALWARE_DIR,
    api_key=ANDROZOO_API_KEY,
    num_samples=NUM_SAMPLES,
    max_workers=MAX_WORKERS
)

# Download the benign APKs
download_apks_parallel(
    filtered_benign_apk_catalog,
    output_dir=BENIGN_DIR,
    api_key=ANDROZOO_API_KEY,
    num_samples=NUM_SAMPLES,
    max_workers=MAX_WORKERS
)

Downloaded: 920C63FEB3012C11370017BA2DEEB61114309066E440D7AE743EEA4D5845F927
Downloaded: 546195D446C593D9ADA3251320AD572CFA2457A184D250F48B86C113974D450F
Downloaded: DD5C00004A3E0626E78D6715AC791138CD4E4032B6506B2886CBE72874BD19E1


Downloaded: DEC2333C58CEB16851F4C8CA49688F8BA0C35151F2FF74956102AC8D294104FF
Downloaded: 20B0E8B49936E6B0EAB70278724449C80B80E9B97C2F21E787BEBA0874C31E7C
Downloaded: 12AB36DB7969D5B0DB7FC7E2BE50A9600024F307BA9B4472CAB5AA748FF6C7D0


Downloaded: 0D384F940E1A7C22A2D18EE9CB3223D2079D153658F29B283E24F3E0C3067FB5
Downloaded: 874F5CF8EB48B9720B15AC779459E99B7A00B8B620AD9D084824F1749A820FB9


Downloaded: E904727EF4AAD2C6D4BCE79D98FABE7E9875EECF1E982141FEDE6A6CE3703B11
Downloaded: FAA0EFAAD40E78BF27CA529171AAF0551DB998A276D4FF501209D1F5EF830DFB


Downloaded: 66A350B76B149CD533E5255F2543DD8CCF87852F96C192B9096B774F36E339E4
Downloaded: 281213A6A64D28627FFD60BAD5332957D0848C56E826FF7D4198ABCEDB1E9A8B
Downloaded: 5C78351DF2C4848D11C27A3621F4A0B8CB59A9D162BC907C408DFF10B6C1EA79


...

Downloaded: C928DA915E9C1850491126B5C77B457E51470646CC187F688F2A8E59623495C8
Downloaded: 2BDBF49D796642D9CE431B8D5C4C2CA02031E8353E9A2D54BCFD246521147D3B
Downloaded: B86C3E2D2B288A25029B53C3CA15C0FA8B6029F88ED8DAE917C0FF50D58AC896
Downloaded: 62EBCA3B34EBCF0E10791D810D8ACA83901ADF01A3B2D1A3F62D8EEFD6896493
Downloaded: F4A9EAE2633D4EDD06C01637162EA56CA07412894CC6D4D6AE4FE0C02999F980


Downloaded: DDFEB7232FD9EF9DA07C7E3A54B2901062B1378BD96A8855719ED12273253C94


Downloaded: 71D89D438AB53BBA29DC750E387B3452DC3CE8850C9F1D90D34388582C57E4EC
Downloaded: 6378A2DAF31764727A670BFBAB9C9A0CC5B41FD870242639981F4FE8D9746056


Downloaded: A8C08C7BCE9EC92442D800F6ACD2B84D3FCB74B1E71DC9739293B04ECB9F3C99

Download Summary:
  Successful: 10000/10000
  Failed: 0/10000


['...',
 ...]